In [51]:
import os, glob
import scipy.io as sio
import numpy as np

In [36]:
#noisy_root = '/home/data/kbh/CHiME4/isolated_1ch_track/'
#estim_root = '/home/data/kbh/CGMM_RLS_MPDR/trial_04/'
#mask_root = '/home/data/kbh/CGMM_RLS_MPDR/trial_04_mask/'

noisy_root = '/home/kiosk/dnn2/CHiME4/isolated_1ch_track/'
estim_root = '/home/kiosk/dnn2/CGMM_RLS_MPDR/trial_04/'
mask_root =  '/home/kiosk/dnn2/CGMM_RLS_MPDR/trial_04_mask/'

In [37]:
dirs_1 = ['dt','et']
dirs_2 = ['bus','caf','str','ped']
dirs_3 = ['real','simu']
dirs =[]
for i in dirs_1 :
    for j in dirs_2 :
        for k in dirs_3 :
            dirs.append(i+'05_'+j+'_'+k)

In [39]:
noisy_list = [x for x in glob.glob(os.path.join(noisy_root,'*','*.wav'))]
estim_list = []
mask_list  = []
for i in dirs : 
    estim_list += [x for x in glob.glob(os.path.join(estim_root,i,'*.wav'))]
    mask_list  += [x for x in glob.glob(os.path.join(mask_root,i,'*.mat'))]
print(len(noisy_list))
print(len(estim_list))
print(len(mask_list))

5920
5920
5920


In [40]:
mat_contents = sio.loadmat(mask_list[0])

In [52]:
print(type(mat_contents))
print(mat_contents)
print(type(mat_contents['noise_mask']))
print(np.shape(mat_contents['noise_mask']))

<class 'dict'>
{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Dec  5 14:28:39 2020', '__version__': '1.0', '__globals__': [], 'noise_mask': array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        3.23045512e-01, 3.84709036e-05, 4.69712414e-03],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        7.12779994e-03, 7.08078041e-12, 2.88657986e-15],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.22044605e-16, 6.66133815e-16, 0.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.58588815e-01, 2.80801547e-01, 1.12558290e-01],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        8.65711918e-01, 7.36254569e-01, 3.96382394e-01],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        9.11370669e-01, 9.11472791e-01, 5.67807477e-01]])}
<class 'numpy.ndarray'>
(513, 435)


In [48]:
type(mat_contents['noise_mask'])

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        3.23045512e-01, 3.84709036e-05, 4.69712414e-03],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        7.12779994e-03, 7.08078041e-12, 2.88657986e-15],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.22044605e-16, 6.66133815e-16, 0.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.58588815e-01, 2.80801547e-01, 1.12558290e-01],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        8.65711918e-01, 7.36254569e-01, 3.96382394e-01],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        9.11370669e-01, 9.11472791e-01, 5.67807477e-01]])

## NOTE
Noise Masking을 하기 위해선 noisy wav를 stft 로 바꾸고 masking하고 다시 wav로 바꿔야한다. 싱크를 잘 맞춰야한다. 

In [53]:
import librosa

In [55]:
print(noisy_list[0])
print(estim_list[0])
print(mask_list[0])

sample_noisy_path = noisy_root + '/dt05_ped_simu/F01_050C0103_PED.wav'
sample_estim_path = estim_root + '/dt05_ped_simu/F01_050C0103_PED.wav'
sample_mask_path = mask_root + '/dt05_ped_simu/F01_050C0103_PED.mat'

/home/kiosk/dnn2/CHiME4/isolated_1ch_track/dt05_ped_simu/F01_050C0103_PED.wav
/home/kiosk/dnn2/CGMM_RLS_MPDR/trial_04/dt05_bus_real/M04_423C020Y_BUS.wav
/home/kiosk/dnn2/CGMM_RLS_MPDR/trial_04_mask/dt05_bus_real/M03_050C0102_BUS.mat


In [60]:
sample_noisy,_ = librosa.load(sample_noisy_path,sr=16000)
sample_estim,_ = librosa.load(sample_estim_path,sr=16000)
sample_mask = sio.loadmat(sample_mask_path)['noise_mask']
print(np.shape(sample_noisy))
print(np.shape(sample_estim))
print(np.shape(sample_mask))

(99810,)
(99810,)
(513, 386)


## NOTE
왜 길이가 같지? STFT overlap 딜레이 고려 안한건가?  
해당 부분에 대해서는 처리된것 같다?

In [61]:
sample_noisy_spec = librosa.stft(sample_noisy,window='hann',n_fft=1024,center=False)
sample_estim_spec = librosa.stft(sample_estim,window='hann',n_fft=1024,center=False)
print(np.shape(sample_noisy_spec))
print(np.shape(sample_estim_spec))

(513, 386)
(513, 386)


In [65]:
x = sample_estim_spec * sample_mask
print(np.shape(x))

(513, 386)


ㄱ